# Generative Adversarial Networks using PyTorch

Here, we implement a basic GAN model using PyTorch to generate handwritten digits.
***

## Import PyTorch packages

Ensure that you have the packages installed on your machine.  
If not, please install the following packages by running: `pip install [PACKAGE]` in your terminal, where [PACKAGE] are:  
torchvision, tensorboardx, jupyter, matplotlib, numpy 

In [1]:
# install packages on your laptop
# run in the terminal (mac): pip install torchvision tensorboardx jupyter matplotlib numpy

# import packages
import torch
from torch import nn, optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets
import torchvision.transforms.functional as F

# record your training precess
from utils import Logger

# python packages
import numpy as np
import matplotlib.pyplot as plt

## Get dataset and perform ETL

1. Get MNIST dataset (handwritten digits) from PyTorch datasets
2. Transform the datasets into tensors and normalize them in range[-1,1]
3. Store them in a local directory './MNIST_dataset'
4. Load the dataset into a PyTorch DataLoader
    - define the batch size as 100 (train on 100 records and labels before making one model iteration)
    - set shuffle as True (shuffle the data for training and test split)
    - format it as a DataLoader to facilitate feeding of input and labels into the model

In [2]:
# get the mnist data
def mnist_data():
    compose = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((.5, .5, .5), (.5, .5, .5))
        ])
    out_dir = './MNIST_dataset'
    return datasets.MNIST(root=out_dir, train=True, transform=compose, download=True)
# Load data
data = mnist_data()
# Create loader with data, so that we can iterate over it
data_loader = torch.utils.data.DataLoader(data, batch_size=100, shuffle=True)
# Num batches
num_batches = len(data_loader)

## Visualize example data records
Plot 5 examples of handwritten digits and their corresponding labels.

In [ ]:
def plottensor(tensor,label):
    fig, ax = plt.subplots(figsize=(2, 2))
    img = F.to_pil_image(tensor)
    ax.imshow(np.asarray(img))
    ax.set_title(label)
    plt.show()

In [ ]:
for batch_idx, (tensors, labels) in enumerate(data_loader):
    stop = 5
    for record_idx, tensor in enumerate(tensors):
        label = str(labels[record_idx])
        plottensor(tensor,label)
        if record_idx ==  stop:
            break
    break

## Define Tensor re-shaping functions
These functions will re-shape tensors between 2 forms:
1. flattened vector of shape 784 (1D-tensor for input into model)
2. square matrix of shape 1x28x28 (single channel / greyscale 28x28 pixels image)

In [3]:
# convert a flattened image into its 2-dimensional representation, 
# and another one that does the opposite.
def images_to_vectors(images):
    return images.view(images.size(0), 784)

def vectors_to_images(vectors):
    return vectors.view(vectors.size(0), 1, 28, 28)

## Discriminator Network

We will construct the discriminator network using PyTorch's neural network module (torch.nn.Module)

The Discriminator will be a 3 hidden layers multi-layer perceptron with the following structure:
        
### Hidden Layer 1 - Fully Connected: linear NN with W and b
        - Input: 1D-tensor (784,) - 28x28 image flattened into a vector
        - Output: 1D-tensor (1084,)
        - Activation Function: Leaky ReLU
        - Regularization: Dropout

### Hidden Layer 2 - Fully Connected: linear NN with W and b
        - Input: 1D-tensor (1084,)
        - Output: 1D-tensor (512,)
        - Activation Function: Leaky ReLU
        - Regularization: Dropout

### Hidden Layer 3 - Fully Connected: linear NN with W and b
        - Input: 1D-tensor (512)
        - Output: 1D-tensor (256)
        - Activation Function: Leaky ReLU
        - Regularization: Dropout

### Output Layer - Fully Connected: linear NN with W and b
        - Input: 1D-tensor (256)
        - Output: 1D-tensor (1) - output indicating probability of image being Real or Fake
        - Activation Function: Sigmoid - squash into a Binary Variable 0 or 1

In [4]:
# build 3 hidden layer nn for discriminator
class DiscriminatorNet(torch.nn.Module):
    """
    A three hidden-layer discriminative neural network
    """
    def __init__(self):
        super(DiscriminatorNet, self).__init__()
        n_features = 784
        n_out = 1
        
        self.hidden0 = nn.Sequential( 
            nn.Linear(n_features, 1024),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        self.hidden1 = nn.Sequential(
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        self.hidden2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        self.out = nn.Sequential(
            torch.nn.Linear(256, n_out),
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        x = self.hidden0(x)
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = self.out(x)
        return x

## Generator Network

We will construct the generator network using PyTorch's neural network module (torch.nn.Module)

The Generator will be a 3 hidden layers multi-layers perceptron with the following structure:
        
### Hidden Layer 1 - Fully Connected: linear NN with W and b
        - Input: 1D-tensor (100,) - noise vector
        - Output: 1D-tensor (256,)
        - Activation Function: Leaky ReLU

### Hidden Layer 2 - Fully Connected: linear NN with W and b
        - Input: 1D-tensor (256,)
        - Output: 1D-tensor (512,)
        - Activation Function: Leaky ReLU

### Hidden Layer 3 - Fully Connected: linear NN with W and b
        - Input: 1D-tensor (512,)
        - Output: 1D-tensor (1024)
        - Activation Function: Leaky ReLU

### Output Layer - Fully Connected: linear NN with W and b
        - Input: 1D-tensor (1024,)
        - Output: 1D-tensor (784,) - 28x28 image flattened into a vector
        - Activation Function: TanH - to ensure value range of each pixel is [-1,1]

In [5]:
class GeneratorNet(torch.nn.Module):
    """
    A three hidden-layer generative neural network
    """
    def __init__(self):
        super(GeneratorNet, self).__init__()
        n_features = 100
        n_out = 784
        
        self.hidden0 = nn.Sequential(
            nn.Linear(n_features, 256),
            nn.LeakyReLU(0.2)
        )
        self.hidden1 = nn.Sequential(            
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2)
        )
        self.hidden2 = nn.Sequential(
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2)
        )
        
        self.out = nn.Sequential(
            nn.Linear(1024, n_out),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.hidden0(x)
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = self.out(x)
        return x


## Define noise-generating function
This function will generate a 1D-tensor of shape (100,) randomly sampled from a gaussian distribution.  
The noise vector will be used as input for the Generator.

In [6]:
def noise(size):
    '''
    Generates a 1-d vector of gaussian sampled random values
    '''
    n = Variable(torch.randn(size, 100))
    return n

## Load models, define optimizer and loss functions

1. We will load our models by creating an object from the DiscriminatorNet and GeneratorNet classes we have defined before.
2. We select the Adam optimizer as our optimization algorithm for both deep learning models.
3. We select the Binary Cross Entropy as our loss function.

In [7]:
# load your model
discriminator = DiscriminatorNet()
generator = GeneratorNet()

# set loss function
d_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002)
g_optimizer = optim.Adam(generator.parameters(), lr=0.0002)
loss = nn.BCELoss()

## Define functions for ground-truth labels
Here we will define two functions for generating a vector of either all 1s or all 0s.  
They will be used as the groud-truth labels for the discriminator.  
Real images will be labled as 1s and fake images as 0s

In [8]:
def ones_target(size):
    '''
    Tensor containing ones, with shape = size
    '''
    data = Variable(torch.ones(size, 1))
    return data

def zeros_target(size):
    '''
    Tensor containing zeros, with shape = size
    '''
    data = Variable(torch.zeros(size, 1))
    return data

## Training the Discriminator

To train the discriminator, we will go through the following steps:

1. Reset gradients (necessary to reset the gradient at each iteration as PyTorch accumulates the gradient)
2. Train the discriminator on real data
    - Make a prediction on the probability of samples being real data
    - calculate the loss from 100 samples of real data
    - compute the gradients by backpropagation using the backward() function
3. Train the discriminator on fake data
    - Make a prediction on the probability of samples being real data
    - calculate the loss from 100 samples of fake data
    - compute the gradients by backpropagation using the backward() function
4. Update the weights of Discriminator using the gradients from the 2 previous steps and the optimizer
5. Return the loss, predictions for logging purposes

In [9]:
# train discriminator
def train_discriminator(optimizer, real_data, fake_data):
    N = real_data.size(0)
    # Reset gradients
    optimizer.zero_grad()
    
    # 1.1 Train on Real Data
    prediction_real = discriminator(real_data)
    # Calculate error and backpropagate
    error_real = loss(prediction_real, ones_target(N) )
    error_real.backward()

    # 1.2 Train on Fake Data
    prediction_fake = discriminator(fake_data)
    # Calculate error and backpropagate
    error_fake = loss(prediction_fake, zeros_target(N))
    error_fake.backward()
    
    # 1.3 Update weights with gradients
    optimizer.step()
    
    # Return error and predictions for real and fake inputs
    return error_real + error_fake, prediction_real, prediction_fake

## Training the Generator

To train the discriminator, we will go through the following steps:

1. Reset gradients (necessary to reset the gradient at each iteration as PyTorch accumulates the gradient)
2. Sample noise vectors and generate fake images using the Generator
3. Use the Discriminator to a prediction on the probability of samples being real data
    - calculate the loss from 100 samples of fake data
    - compute the gradients by backpropagation using the backward() function
4. Update the weights of Generator using the gradients from the 2 previous steps and the optimizer
5. Return the loss for logging purposes

In [10]:
# train generator
def train_generator(optimizer, fake_data):
    N = fake_data.size(0)
    # Reset gradients
    optimizer.zero_grad()
    # Sample noise and generate fake data
    prediction = discriminator(fake_data)
    # Calculate error and backpropagate
    error = loss(prediction, ones_target(N))
    error.backward()
    # Update weights with gradients
    optimizer.step()
    # Return error
    return error

# Logger, samples and epoch

Here we will define 3 things:
1. The number of generated image samples that the script will spit out after every batch of training, so we can keep track of the progress.
2. The logger instance that will print out the training progress as well as the loss and prediction accuracy of the Generator and Discriminator.
3. The number of Epochs (the number of complete forward and backward pass of all the training examples).

In [11]:
num_test_samples = 16
test_noise = noise(num_test_samples)

# Create logger instance
logger = Logger(model_name='VGAN', data_name='MNIST')
# Total number of epochs to train
num_epochs = 200

## Main block of code to train GAN

For each epoch of 200 epochs:
    For each batch of 100 examples in the training dataset:
        1. Train the Discriminator
            a. Transform the batch of 100 examples into a 1D-tensors of shape (784,)
            b. Detach the Generator so the gradients are not calculated for the Generator
            c. Train the Discriminator first using a combination of real and fake data
        2. Train the Generator
            a. Generate a batch of 100 fake examples by inputting the noise vectors into the Generator
            b. Train the Generator using the fake data on the Discriminator
        3. For every 100 batches of training, print the following:
            a. Current epoch number, batch number
            b. Discriminator and Generator loss
            c. Prediction of Discriminator on real and fake examples (between [0,1])

In [20]:
for epoch in range(num_epochs):
    for n_batch, (real_batch,_) in enumerate(data_loader):
        N = real_batch.size(0)
        # 1. Train Discriminator
        real_data = Variable(images_to_vectors(real_batch))
        # Generate fake data and detach 
        # (so gradients are not calculated for generator)
        fake_data = generator(noise(N)).detach()
        # Train D
        d_error, d_pred_real, d_pred_fake = train_discriminator(d_optimizer, real_data, fake_data)

        # 2. Train Generator
        # Generate fake data
        fake_data = generator(noise(N))
        # Train G
        g_error = train_generator(g_optimizer, fake_data)
        
        # Log batch error
        logger.log(d_error, g_error, epoch, n_batch, num_batches)
        print(float(d_error))
        print(d_error.detach().numpy())
        print(g_error)
        # Display Progress every few batches
        if (n_batch) % 100 == 0: 
            # just change class tensor to numeric for easy printing
            d_error = float(d_error)
            g_error = float(d_error)
            test_images = vectors_to_images(generator(test_noise))
            test_images = test_images.data
            .log_images(
                test_images, num_test_samples, 
                epoch, n_batch, num_batches
            )
            # Display status Logs
            logger.display_status(
                epoch, num_epochs, n_batch, num_batches,
                d_error, g_error, d_pred_real, d_pred_fake
            )

logger.save_models()            

0.9239945411682129
0.9239945411682129
tensor(0.5258, grad_fn=<BinaryCrossEntropyBackward>)


Epoch: [0/200], Batch Num: [0/600]
Discriminator Loss: 0.9240, Generator Loss: 0.9240
D(x): 0.9622, D(G(z)): 0.5871
0.9971814751625061
0.9971814751625061
tensor(0.4727, grad_fn=<BinaryCrossEntropyBackward>)
1.0923625230789185
1.0923625230789185
tensor(0.4296, grad_fn=<BinaryCrossEntropyBackward>)
1.1927592754364014
1.1927592754364014
tensor(0.3935, grad_fn=<BinaryCrossEntropyBackward>)
1.2831720113754272
1.2831720113754272
tensor(0.3671, grad_fn=<BinaryCrossEntropyBackward>)
1.3396048545837402
1.3396048545837402
tensor(0.3620, grad_fn=<BinaryCrossEntropyBackward>)
1.369153380393982
1.369153380393982
tensor(0.3660, grad_fn=<BinaryCrossEntropyBackward>)
1.3581434488296509
1.3581434488296509
tensor(0.4006, grad_fn=<BinaryCrossEntropyBackward>)
1.2943698167800903
1.2943698167800903
tensor(0.4534, grad_fn=<BinaryCrossEntropyBackward>)
1.244057536125183
1.244057536125183
tensor(0.5296, grad_fn=<BinaryCrossEntropyBackward>)
1.1638450622558594
1.1638450622558594
tensor(0.6504, grad_fn=<BinaryC

Epoch: [0/200], Batch Num: [100/600]
Discriminator Loss: 0.2137, Generator Loss: 0.2137
D(x): 0.9589, D(G(z)): 0.1513
0.17653480172157288
0.17653480172157288
tensor(3.1064, grad_fn=<BinaryCrossEntropyBackward>)
0.16570281982421875
0.16570281982421875
tensor(3.3454, grad_fn=<BinaryCrossEntropyBackward>)
0.17495596408843994
0.17495596408843994
tensor(3.4505, grad_fn=<BinaryCrossEntropyBackward>)
0.16453836858272552
0.16453836858272552
tensor(3.3375, grad_fn=<BinaryCrossEntropyBackward>)
0.25341153144836426
0.25341153144836426
tensor(3.0204, grad_fn=<BinaryCrossEntropyBackward>)
0.3618582487106323
0.3618582487106323
tensor(2.5811, grad_fn=<BinaryCrossEntropyBackward>)
0.5606268644332886
0.5606268644332886
tensor(2.1131, grad_fn=<BinaryCrossEntropyBackward>)
0.5954033136367798
0.5954033136367798
tensor(1.8229, grad_fn=<BinaryCrossEntropyBackward>)
0.5536496639251709
0.5536496639251709
tensor(2.0725, grad_fn=<BinaryCrossEntropyBackward>)
0.5061486959457397
0.5061486959457397
tensor(2.0155, 

Epoch: [0/200], Batch Num: [200/600]
Discriminator Loss: 0.4415, Generator Loss: 0.4415
D(x): 0.8253, D(G(z)): 0.1739
0.4439695477485657
0.4439695477485657
tensor(3.4637, grad_fn=<BinaryCrossEntropyBackward>)
0.5188238620758057
0.5188238620758057
tensor(3.3801, grad_fn=<BinaryCrossEntropyBackward>)
0.4830707311630249
0.4830707311630249
tensor(3.4810, grad_fn=<BinaryCrossEntropyBackward>)
0.7137166261672974
0.7137166261672974
tensor(3.1903, grad_fn=<BinaryCrossEntropyBackward>)
0.7435145378112793
0.7435145378112793
tensor(3.2143, grad_fn=<BinaryCrossEntropyBackward>)
0.8543281555175781
0.8543281555175781
tensor(2.8258, grad_fn=<BinaryCrossEntropyBackward>)
1.013941764831543
1.013941764831543
tensor(2.8448, grad_fn=<BinaryCrossEntropyBackward>)
1.074245810508728
1.074245810508728
tensor(2.6253, grad_fn=<BinaryCrossEntropyBackward>)
1.2574056386947632
1.2574056386947632
tensor(2.7054, grad_fn=<BinaryCrossEntropyBackward>)
1.185136079788208
1.185136079788208
tensor(2.8281, grad_fn=<BinaryC

Epoch: [0/200], Batch Num: [300/600]
Discriminator Loss: 0.0887, Generator Loss: 0.0887
D(x): 0.9642, D(G(z)): 0.0248
0.06513553112745285
0.06513553112745285
tensor(6.8210, grad_fn=<BinaryCrossEntropyBackward>)
0.09705138206481934
0.09705138206481934
tensor(6.5561, grad_fn=<BinaryCrossEntropyBackward>)
0.15632043778896332
0.15632043778896332
tensor(6.3050, grad_fn=<BinaryCrossEntropyBackward>)
0.16838020086288452
0.16838020086288452
tensor(7.3246, grad_fn=<BinaryCrossEntropyBackward>)
0.1508169174194336
0.1508169174194336
tensor(7.5190, grad_fn=<BinaryCrossEntropyBackward>)
0.15784917771816254
0.15784917771816254
tensor(8.7425, grad_fn=<BinaryCrossEntropyBackward>)
0.12869831919670105
0.12869831919670105
tensor(8.2673, grad_fn=<BinaryCrossEntropyBackward>)
0.3137562870979309
0.3137562870979309
tensor(7.8338, grad_fn=<BinaryCrossEntropyBackward>)
0.2108864188194275
0.2108864188194275
tensor(7.1892, grad_fn=<BinaryCrossEntropyBackward>)
0.4019615650177002
0.4019615650177002
tensor(6.7824

...

Epoch: [3/200], Batch Num: [0/600]
Discriminator Loss: 0.5489, Generator Loss: 0.5489
D(x): 0.8554, D(G(z)): 0.2046
0.4781779944896698
0.4781779944896698
tensor(2.8618, grad_fn=<BinaryCrossEntropyBackward>)
0.5868210196495056
0.5868210196495056
tensor(3.1454, grad_fn=<BinaryCrossEntropyBackward>)
0.3405907154083252
0.3405907154083252
tensor(3.5655, grad_fn=<BinaryCrossEntropyBackward>)
0.5333925485610962
0.5333925485610962
tensor(3.5539, grad_fn=<BinaryCrossEntropyBackward>)
0.3382234275341034
0.3382234275341034
tensor(3.4524, grad_fn=<BinaryCrossEntropyBackward>)
0.2573990821838379
0.2573990821838379
tensor(3.6994, grad_fn=<BinaryCrossEntropyBackward>)
0.15246403217315674
0.15246403217315674
tensor(4.3895, grad_fn=<BinaryCrossEntropyBackward>)
0.14411191642284393
0.14411191642284393
tensor(5.0775, grad_fn=<BinaryCrossEntropyBackward>)
0.13336101174354553
0.13336101174354553
tensor(5.9757, grad_fn=<BinaryCrossEntropyBackward>)
0.1381586790084839
0.1381586790084839
tensor(6.7084, grad_f

Epoch: [3/200], Batch Num: [100/600]
Discriminator Loss: 0.4968, Generator Loss: 0.4968
D(x): 0.8692, D(G(z)): 0.2263
0.6626458168029785
0.6626458168029785
tensor(2.7922, grad_fn=<BinaryCrossEntropyBackward>)
0.6716258525848389
0.6716258525848389
tensor(2.4901, grad_fn=<BinaryCrossEntropyBackward>)
0.4605143964290619
0.4605143964290619
tensor(2.1763, grad_fn=<BinaryCrossEntropyBackward>)
0.6841481924057007
0.6841481924057007
tensor(2.0303, grad_fn=<BinaryCrossEntropyBackward>)
0.7598563432693481
0.7598563432693481
tensor(2.1962, grad_fn=<BinaryCrossEntropyBackward>)
0.8458033204078674
0.8458033204078674
tensor(2.3880, grad_fn=<BinaryCrossEntropyBackward>)
0.6223130226135254
0.6223130226135254
tensor(2.8482, grad_fn=<BinaryCrossEntropyBackward>)
0.7785521149635315
0.7785521149635315
tensor(2.6909, grad_fn=<BinaryCrossEntropyBackward>)
0.7043042182922363
0.7043042182922363
tensor(2.1393, grad_fn=<BinaryCrossEntropyBackward>)
0.6981357932090759
0.6981357932090759
tensor(2.0962, grad_fn=<B

Epoch: [3/200], Batch Num: [200/600]
Discriminator Loss: 0.6873, Generator Loss: 0.6873
D(x): 0.7599, D(G(z)): 0.1731
0.6462019681930542
0.6462019681930542
tensor(2.0629, grad_fn=<BinaryCrossEntropyBackward>)
0.7327267527580261
0.7327267527580261
tensor(2.0470, grad_fn=<BinaryCrossEntropyBackward>)
0.631920337677002
0.631920337677002
tensor(2.3464, grad_fn=<BinaryCrossEntropyBackward>)
0.8430567979812622
0.8430567979812622
tensor(2.7161, grad_fn=<BinaryCrossEntropyBackward>)
0.7540631294250488
0.7540631294250488
tensor(2.5002, grad_fn=<BinaryCrossEntropyBackward>)
0.6404979825019836
0.6404979825019836
tensor(2.1685, grad_fn=<BinaryCrossEntropyBackward>)
0.6560766100883484
0.6560766100883484
tensor(2.3830, grad_fn=<BinaryCrossEntropyBackward>)
0.5221085548400879
0.5221085548400879
tensor(2.4571, grad_fn=<BinaryCrossEntropyBackward>)
0.5574265718460083
0.5574265718460083
tensor(2.6042, grad_fn=<BinaryCrossEntropyBackward>)
0.3403139114379883
0.3403139114379883
tensor(3.5725, grad_fn=<Bin

Epoch: [3/200], Batch Num: [300/600]
Discriminator Loss: 0.7912, Generator Loss: 0.7912
D(x): 0.7709, D(G(z)): 0.2073
0.4842763841152191
0.4842763841152191
tensor(3.5368, grad_fn=<BinaryCrossEntropyBackward>)
0.5738463401794434
0.5738463401794434
tensor(3.3726, grad_fn=<BinaryCrossEntropyBackward>)
0.5648890733718872
0.5648890733718872
tensor(3.6253, grad_fn=<BinaryCrossEntropyBackward>)
0.4507090449333191
0.4507090449333191
tensor(3.3048, grad_fn=<BinaryCrossEntropyBackward>)
0.31653475761413574
0.31653475761413574
tensor(2.9210, grad_fn=<BinaryCrossEntropyBackward>)
0.5024883151054382
0.5024883151054382
tensor(3.1955, grad_fn=<BinaryCrossEntropyBackward>)
0.3039925694465637
0.3039925694465637
tensor(3.2745, grad_fn=<BinaryCrossEntropyBackward>)
0.3320464491844177
0.3320464491844177
tensor(3.4015, grad_fn=<BinaryCrossEntropyBackward>)
0.24476942420005798
0.24476942420005798
tensor(3.5669, grad_fn=<BinaryCrossEntropyBackward>)
0.3346586227416992
0.3346586227416992
tensor(4.0075, grad_f

KeyboardInterrupt: 